In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
from datetime import datetime

d:\python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [4]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
#     print('y_pre.shape:',y_pre.shape)   # 这个求出来一堆概率
#     print(tf.argmax(y_pre,1))    # 这个就是找出概率这一列中取值最大的那个，也就是我要取得值。
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result
def weight_variable(shape):
    inital = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(inital) 
def bias_variable(shape):
    inital = tf.constant(0.1,shape=shape)##初始值为0.1
    return tf.Variable(inital)  
def conv2d(x,W):
    ##strides=[batch,height,width,channel]，所以固定不动Batch和channel,只改变height和width的步长
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME') 
def max_pool_2(x):
    ###kszie难道不是指核的大小？？？不是！！！！
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')  

## conv1 layer ##
def model(x_image):
    W_conv1 = weight_variable([5,5,1,32]) ##patch5*5,in_size=1,out_size=32
    b_conv1 = bias_variable([32])
    conv1 = tf.nn.bias_add(conv2d(x_image,W_conv1),b_conv1)  ##output:28*28*32
    conv1_relu = tf.nn.relu(conv1)   ##output:28*28*32
    conv1_pool = max_pool_2(conv1_relu)  ##output:14*14*32

    ## conv2 layer ##
    W_conv2 = weight_variable([5,5,32,64]) ##patch5*5,in_size=32,out_size=64
    b_conv2 = bias_variable([64])
    conv2 = tf.nn.bias_add(conv2d(conv1_pool,W_conv2),b_conv2)  ##output:14*14*64
    conv2_relu = tf.nn.relu(conv2)   ##output:14*14*64
    conv2_pool = max_pool_2(conv2_relu)  ##output:7*7*64

    ##func1 layer##
    W_fc1 = weight_variable([7*7*64,1024])
    b_fc1 = bias_variable([1024])
    ##shape[n_sample,7,7,64]-->>[n_sample,7*7*64]
    conv2_pool_flat = tf.reshape(conv2_pool,[-1,7*7*64])
    fc1 = tf.matmul(conv2_pool_flat,W_fc1)+b_fc1
    fc1_relu = tf.nn.relu(fc1)
    fc1_relu_drop = tf.nn.dropout(fc1_relu,keep_prob)

    ##func2 layer##
    W_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])
    fc2 = tf.matmul(fc1_relu_drop,W_fc2)+b_fc2
    # prediction = tf.nn.softmax(fc2)
    ##fc2_relu_drop = tf.nn.dropout(fc2_relu,keep_Prob)
    return fc2

In [27]:
j = 1
with tf.Graph().as_default():
        ###define placeholder for inputs to network
        xs = tf.placeholder(tf.float32,[None,784])  ##28*28，None表示其值大小不定，在这里作为第一个维度值，用以指代batch的大小，意即x的数量不定
        ys = tf.placeholder(tf.float32,[None,10])  ##10类输出
        keep_prob = tf.placeholder(tf.float32)  ###定义丢掉率

        x_image = tf.reshape(xs,[-1,28,28,1])##shape:[n_sample,28,28,1]

        prediction = model(x_image)
        global_steps = tf.Variable(1, trainable=False)

        # the error between prediction and real data
    #     cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))       # loss
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=ys))
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy,global_step=global_steps)

        saver = tf.train.Saver()

        format_str = "%s    Iteration =  %d   loss = %0.3f"
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            for i in range(20+1):
                batch_xs,batch_ys = mnist.train.next_batch(100)
                loss_val,_ = sess.run([cross_entropy,train_step],feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.5})

                if i % 10 == 0:
                    print('loss:',format_str%(datetime.now(),i,loss_val))
    #                 print('正确率：',compute_accuracy(mnist.test.images,mnist.test.labels))

                if i % 20 == 0:
                    file = str(j)+"saver_手写数字"
                    checkpoint = file+'/' + 'model.ckpt'
                    saver.save(sess, checkpoint,global_step=global_steps)
print('训练结束！')

loss: 2018-12-21 20:32:20.086679    Iteration =  0   loss = 10.821
loss: 2018-12-21 20:32:22.530500    Iteration =  10   loss = 6.792
loss: 2018-12-21 20:32:24.364703    Iteration =  20   loss = 4.512
训练结束！


In [32]:
def train():
    j = 0
    j = j+1
    with tf.Graph().as_default():
        ###define placeholder for inputs to network
        xs = tf.placeholder(tf.float32,[None,784])  ##28*28，None表示其值大小不定，在这里作为第一个维度值，用以指代batch的大小，意即x的数量不定
        ys = tf.placeholder(tf.float32,[None,10])  ##10类输出
        keep_prob = tf.placeholder(tf.float32)  ###定义丢掉率
        x_image = tf.reshape(xs,[-1,28,28,1])##shape:[n_sample,28,28,1]
        prediction = model(x_image)
        global_steps = tf.Variable(1, trainable=False)
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=ys))
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy,global_step=global_steps)
        saver = tf.train.Saver()
        format_str = "%s    Iteration =  %d   loss = %0.3f"
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(20+1):
                batch_xs,batch_ys = mnist.train.next_batch(100)
                loss_val,_ = sess.run([cross_entropy,train_step],feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.5})
                if i % 10 == 0:
                    print('loss:',format_str%(datetime.now(),i,loss_val))
                if i % 20 == 0:
                    file = str(j)+"saver_手写数字"
                    checkpoint = file + 'model.ckpt'
                    saver.save(sess, checkpoint,global_step=global_steps)
#     print('第几次训练结束：','j')

In [ ]:
if __name__ == '__main__':
    j = 0
    train()

## 定时操作测试

In [36]:
def model1(X):
    W1 = tf.get_variable('W1',[784,512],initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable('b1',[512],initializer=tf.zeros_initializer())
    W2 = tf.get_variable('W2',[512,128],initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable('b2',[128],initializer=tf.zeros_initializer())
    W3 = tf.get_variable('W3',[128,64],initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable('b3',[64],initializer=tf.zeros_initializer())
    W4 = tf.get_variable('W4',[64,10],initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.get_variable('b4',[10],initializer=tf.zeros_initializer())

    # 计算网络中间的logits（就是我们常用的Z）、激活值：全部都是全链接的关系
    A1 = tf.nn.relu(tf.matmul(X,W1)+b1,name='A1')   
    A2 = tf.nn.relu(tf.matmul(A1,W2)+b2,name='A2')
    A3 = tf.nn.relu(tf.matmul(A2,W3)+b3,name='A3')
    Z4 = tf.matmul(A3,W4)+b4   # 输出一定是未激活的函数
    return Z4

In [45]:
def train(j):
    with tf.Graph().as_default():
        x = tf.placeholder(tf.float32,[None,784])  ##28*28，None表示其值大小不定，在这里作为第一个维度值，用以指代batch的大小，意即x的数量不定
        y = tf.placeholder(tf.float32,[None,10])  ##10类输出
        global_steps = tf.Variable(1, trainable=False)
        prediction = model1(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cost,global_step=global_steps)
        saver = tf.train.Saver()

        format_str = "%s    Iteration =  %d   loss = %0.3f"
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(100+1):
                batch_x,batch_y = mnist.train.next_batch(batch_size=100)
                loss_val,_ = sess.run([cost,train_step],feed_dict={x:batch_x,y:batch_y})
                if i % 20 == 0:
                    print('loss_val:',format_str%(datetime.now(), i, loss_val))
                if i % 100 == 0:
                    file = str(j)+"saver_手写数字"
                    checkpoint = file +'/'+ 'model.ckpt'
                    saver.save(sess, checkpoint,global_step=global_steps)  
    print('第几次训练结束：',j)

In [50]:
import os
import time
def print_ts(message):
    print ("[%s] %s"%(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), message))
def run_1(interval,j):
    print_ts("-"*100)
    print_ts("Starting every %s seconds."%interval)
    print_ts("-"*100)
    while True:
        try:
            # sleep for the remaining seconds of interval
            train(j)
            j = j+1
            time_remaining = interval-time.time()%interval
            print_ts("Sleeping until %s (%s seconds)..."%((time.ctime(time.time()+time_remaining)), time_remaining))
            time.sleep(time_remaining)
            if j >3:
                print('已经训练了3次，我要结束！')
                break
           
        except Exception as e:
            print (e)

In [51]:
if __name__ == '__main__':
    interval = 10
    run_1(interval,1)

[2018-12-21 20:57:54] ----------------------------------------------------------------------------------------------------
[2018-12-21 20:57:54] Starting every 10 seconds.
[2018-12-21 20:57:54] ----------------------------------------------------------------------------------------------------
loss_val: 2018-12-21 20:57:54.665365    Iteration =  0   loss = 2.342
loss_val: 2018-12-21 20:57:54.921092    Iteration =  20   loss = 1.666
loss_val: 2018-12-21 20:57:55.004854    Iteration =  40   loss = 0.923
loss_val: 2018-12-21 20:57:55.085638    Iteration =  60   loss = 0.479
loss_val: 2018-12-21 20:57:55.189362    Iteration =  80   loss = 0.436
loss_val: 2018-12-21 20:57:55.272139    Iteration =  100   loss = 0.331
第几次训练结束： 1
[2018-12-21 20:57:55] Sleeping until Fri Dec 21 20:58:00 2018 (4.562821388244629 seconds)...
loss_val: 2018-12-21 20:58:00.201607    Iteration =  0   loss = 2.334
loss_val: 2018-12-21 20:58:00.538762    Iteration =  20   loss = 1.716
loss_val: 2018-12-21 20:58:00.6217